<a href="https://colab.research.google.com/github/Zuhair0000/FYP-Research/blob/main/FYP_research.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import os

# List all files in the Colab working directory
print(os.listdir("/content"))

['.config', '风力秋季发电.xlsx', '风力冬季发电.xlsx', '光伏秋季发电.xlsx', '光伏春季发电.xlsx', '光伏冬季发电.xlsx', '光伏夏季发电.xlsx', '风力春季发电.xlsx', '风力夏季发电.xlsx', 'sample_data']


In [ ]:
import glob

all_files = glob.glob("/content/*.xlsx")
print("All Excel files:", all_files)

# Separate solar and wind
solar_files = [f for f in all_files if "光伏" in f]
wind_files  = [f for f in all_files if "风力" in f]

print("Solar files:", solar_files)
print("Wind files:", wind_files)

All Excel files: ['/content/风力秋季发电.xlsx', '/content/风力冬季发电.xlsx', '/content/光伏秋季发电.xlsx', '/content/光伏春季发电.xlsx', '/content/光伏冬季发电.xlsx', '/content/光伏夏季发电.xlsx', '/content/风力春季发电.xlsx', '/content/风力夏季发电.xlsx']
Solar files: ['/content/光伏秋季发电.xlsx', '/content/光伏春季发电.xlsx', '/content/光伏冬季发电.xlsx', '/content/光伏夏季发电.xlsx']
Wind files: ['/content/风力秋季发电.xlsx', '/content/风力冬季发电.xlsx', '/content/风力春季发电.xlsx', '/content/风力夏季发电.xlsx']


In [ ]:
import pandas as pd

# Load solar
solar_dfs = [pd.read_excel(f) for f in solar_files]
solar_data = pd.concat(solar_dfs, ignore_index=True)

# Load wind
wind_dfs = [pd.read_excel(f) for f in wind_files]
wind_data = pd.concat(wind_dfs, ignore_index=True)

print("Solar shape:", solar_data.shape)
print("Wind shape:", wind_data.shape)

Solar shape: (8753, 7)
Wind shape: (8752, 5)


In [ ]:
print("Solar columns:", solar_data.columns)
print(solar_data.head())

print("\nWind columns:", wind_data.columns)
print(wind_data.head())

print("\nMissing values in Solar:\n", solar_data.isnull().sum())
print("\nMissing values in Wind:\n", wind_data.isnull().sum())

Solar columns: Index(['年份', '时间', '温度', '湿度', '地面辐射强度', '大气上层辐射强度', '光伏发电'], dtype='object')
                   年份  时间      温度     湿度  地面辐射强度  大气上层辐射强度  光伏发电
0 2019-09-01 00:00:00   1  11.281  0.009   0.000     0.000   0.0
1 2019-09-01 01:00:00   2  10.447  0.009   0.000     0.000   0.0
2 2019-09-01 02:00:00   3   9.970  0.009   0.000     0.000   0.0
3 2019-09-01 03:00:00   4   9.942  0.009   0.000     0.000   0.0
4 2019-09-01 04:00:00   5   9.962  0.009   2.222     7.152   0.0

Wind columns: Index(['年份', '时间', '空气密度', '风速', '发电功率'], dtype='object')
                   年份  时间   空气密度     风速      发电功率
0 2019-09-01 00:00:00   1  1.166  5.636   916.774
1 2019-09-01 01:00:00   2  1.160  6.993  1703.133
2 2019-09-01 02:00:00   3  1.155  7.430  1989.040
3 2019-09-01 03:00:00   4  1.152  7.637  2135.031
4 2019-09-01 04:00:00   5  1.149  7.548  2069.010

Missing values in Solar:
 年份          0
时间          0
温度          0
湿度          0
地面辐射强度      0
大气上层辐射强度    0
光伏发电        0
dtype: int64

Missi

In [ ]:
# --- Solar ---
solar_data = solar_data.rename(columns={
    "年份": "DateTime",
    "时间": "Index",
    "温度": "Temperature",
    "湿度": "Humidity",
    "地面辐射强度": "Ground_Irradiance",
    "大气上层辐射强度": "Atmospheric_Irradiance",
    "光伏发电": "Solar_Power"
})

# --- Wind ---
wind_data = wind_data.rename(columns={
    "年份": "DateTime",
    "时间": "Index",
    "空气密度": "Air_Density",
    "风速": "Wind_Speed",
    "发电功率": "Wind_Power"
})

In [ ]:
solar_data["DateTime"] = pd.to_datetime(solar_data["DateTime"])
wind_data["DateTime"]  = pd.to_datetime(wind_data["DateTime"])

# Sort by datetime
solar_data = solar_data.sort_values("DateTime")
wind_data  = wind_data.sort_values("DateTime")
solar_data


,DateTime,Index,Temperature,Humidity,Ground_Irradiance,Atmospheric_Irradiance,Solar_Power
5137,2019-01-01 08:00:00,745,-19.889,0.001,82.680,209.252,257.690
5138,2019-01-01 09:00:00,746,-19.004,0.001,149.553,345.604,530.391
5139,2019-01-01 10:00:00,747,-17.971,0.001,193.625,430.654,871.016
5140,2019-01-01 11:00:00,748,-17.281,0.001,203.808,458.550,1087.304
5141,2019-01-01 12:00:00,749,-16.738,0.001,185.193,427.391,1010.161
...,...,...,...,...,...,...,...
5132,2019-12-31 19:00:00,740,-23.621,0.000,0.000,0.000,0.000
5133,2019-12-31 20:00:00,741,-23.980,0.000,0.000,0.000,0.000
5134,2019-12-31 21:00:00,742,-24.826,0.000,0.000,0.000,0.000
5135,2019-12-31 22:00:00,743,-26.172,0.000,0.000,0.000,0.000


In [ ]:
solar_data = solar_data.drop(columns=["Index"])
wind_data  = wind_data.drop(columns=["Index"])

In [ ]:
print("Solar date range:", solar_data["DateTime"].min(), "→", solar_data["DateTime"].max())
print("Wind date range:", wind_data["DateTime"].min(), "→", wind_data["DateTime"].max())

print("\nSolar frequency check:")
print(solar_data["DateTime"].diff().value_counts().head())

print("\nWind frequency check:")
print(wind_data["DateTime"].diff().value_counts().head())

Solar date range: 2019-01-01 08:00:00 → 2019-12-31 23:00:00
Wind date range: 2019-01-01 08:00:00 → 2019-12-31 23:00:00

Solar frequency check:
DateTime
0 days 01:00:00    8751
0 days 00:00:00       1
Name: count, dtype: int64

Wind frequency check:
DateTime
0 days 01:00:00    8751
Name: count, dtype: int64


In [ ]:
# Resample Solar
solar_data = solar_data.set_index("DateTime").resample("H").mean().interpolate()

# Resample Wind
wind_data = wind_data.set_index("DateTime").resample("H").mean().interpolate()

# Reset index back
solar_data = solar_data.reset_index()
wind_data  = wind_data.reset_index()

/tmp/ipython-input-997126668.py:2: FutureWarning: 'H' is deprecated and will be removed in a future version, please use 'h' instead.
  solar_data = solar_data.set_index("DateTime").resample("H").mean().interpolate()
/tmp/ipython-input-997126668.py:5: FutureWarning: 'H' is deprecated and will be removed in a future version, please use 'h' instead.
  wind_data = wind_data.set_index("DateTime").resample("H").mean().interpolate()


In [ ]:
merged_data = pd.merge_asof(
    solar_data.sort_values("DateTime"),
    wind_data.sort_values("DateTime"),
    on="DateTime",
    direction="nearest"  # matches closest timestamp
)
merged_data

,DateTime,Temperature,Humidity,Ground_Irradiance,Atmospheric_Irradiance,Solar_Power,Air_Density,Wind_Speed,Wind_Power
0,2019-01-01 08:00:00,-19.889,0.001,82.680,209.252,257.690,1.335,6.178,1203.800
1,2019-01-01 09:00:00,-19.004,0.001,149.553,345.604,530.391,1.337,6.534,1410.415
2,2019-01-01 10:00:00,-17.971,0.001,193.625,430.654,871.016,1.339,6.747,1547.633
3,2019-01-01 11:00:00,-17.281,0.001,203.808,458.550,1087.304,1.340,6.765,1556.072
4,2019-01-01 12:00:00,-16.738,0.001,185.193,427.391,1010.161,1.342,6.606,1456.741
...,...,...,...,...,...,...,...,...,...
8747,2019-12-31 19:00:00,-23.621,0.000,0.000,0.000,0.000,1.385,6.260,1254.654
8748,2019-12-31 20:00:00,-23.980,0.000,0.000,0.000,0.000,1.387,5.784,995.959
8749,2019-12-31 21:00:00,-24.826,0.000,0.000,0.000,0.000,1.388,4.955,614.468
8750,2019-12-31 22:00:00,-26.172,0.000,0.000,0.000,0.000,1.388,4.525,457.358


In [ ]:
for lag in [1, 3, 6, 24]:  # 1h, 3h, 6h, 24h lags
    merged_data[f"Solar_Power_lag{lag}"] = merged_data["Solar_Power"].shift(lag)
    merged_data[f"Wind_Power_lag{lag}"] = merged_data["Wind_Power"].shift(lag)

In [ ]:
merged_data["Solar_Power_roll6h"] = merged_data["Solar_Power"].rolling(window=6).mean()
merged_data["Wind_Power_roll6h"] = merged_data["Wind_Power"].rolling(window=6).mean()

In [ ]:
merged_data["hour"] = merged_data["DateTime"].dt.hour
merged_data["dayofweek"] = merged_data["DateTime"].dt.dayofweek
merged_data["month"] = merged_data["DateTime"].dt.month

In [ ]:
merged_data = merged_data.dropna().reset_index(drop=True)

In [ ]:
from sklearn.preprocessing import MinMaxScaler

scaler = MinMaxScaler()
scaled_cols = ["Temperature", "Humidity", "Ground_Irradiance", "Atmospheric_Irradiance",
               "Solar_Power", "Air_Density", "Wind_Speed", "Wind_Power"]

merged_data[scaled_cols] = scaler.fit_transform(merged_data[scaled_cols])

In [ ]:
merged_data

,DateTime,Temperature,Humidity,Ground_Irradiance,Atmospheric_Irradiance,Solar_Power,Air_Density,Wind_Speed,Wind_Power,Solar_Power_lag1,...,Wind_Power_lag3,Solar_Power_lag6,Wind_Power_lag6,Solar_Power_lag24,Wind_Power_lag24,Solar_Power_roll6h,Wind_Power_roll6h,hour,dayofweek,month
0,2019-01-02 08:00:00,0.226369,0.052632,0.118687,0.176241,0.331661,0.778912,0.104728,0.002454,218.077,...,9.707,0.000,49.052,257.690,1203.800,285.091667,14.552000,8,2,1
1,2019-01-02 09:00:00,0.260840,0.052632,0.239731,0.290260,0.679224,0.789116,0.153663,0.017956,1492.473,...,11.803,0.000,28.576,530.391,1410.415,794.509500,24.622500,9,2,1
2,2019-01-02 10:00:00,0.291726,0.052632,0.329584,0.361375,0.776142,0.799320,0.215482,0.067739,3056.507,...,11.997,0.000,13.065,871.016,1547.633,1376.616333,78.403833,10,2,1
3,2019-01-02 11:00:00,0.312578,0.052632,0.359166,0.384702,0.777745,0.806122,0.253304,0.113424,3492.641,...,12.164,0.000,9.707,1087.304,1556.072,1959.925000,170.485000,11,2,1
4,2019-01-02 12:00:00,0.324270,0.052632,0.329195,0.358657,0.751873,0.812925,0.275090,0.145711,3499.852,...,88.999,0.000,11.803,1010.161,1456.741,2523.829667,288.888833,12,2,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8723,2019-12-31 19:00:00,0.156743,0.000000,0.000000,0.000000,0.000000,0.942177,0.332873,0.253130,0.000,...,1405.135,1409.988,1088.591,0.000,1629.479,173.893667,1264.816167,19,1,12
8724,2019-12-31 20:00:00,0.151024,0.000000,0.000000,0.000000,0.000000,0.948980,0.306552,0.200938,0.000,...,1444.480,906.648,978.892,0.000,1374.040,22.785667,1267.660667,20,1,12
8725,2019-12-31 21:00:00,0.137548,0.000000,0.000000,0.000000,0.000000,0.952381,0.260713,0.123971,0.000,...,1386.523,136.714,1119.213,0.000,1468.884,0.000000,1183.536500,21,1,12
8726,2019-12-31 22:00:00,0.116108,0.000000,0.000000,0.000000,0.000000,0.952381,0.236937,0.092273,0.000,...,1254.654,0.000,1405.135,0.000,1482.002,0.000000,1025.573667,22,1,12


In [ ]:
# Save as CSV
merged_data.to_csv("clean_solar_wind.csv", index=False)

# Save as Pickle (keeps data types, useful for reloading later)
merged_data.to_pickle("clean_solar_wind.pkl")

In [ ]:
from google.colab import files
files.download("clean_solar_wind.csv")

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [1]:
from google.colab import output
!git config --global user.email "zuhairhassan52@gmail.com"
!git config --global user.name "Zuhair0000"

In [3]:
!git clone https://github.com/Zuhair0000/FYP-Research

Cloning into 'FYP-Research'...
remote: Enumerating objects: 3, done.
remote: Counting objects: 100% (3/3), done.
remote: Total 3 (delta 0), reused 0 (delta 0), pack-reused 0 (from 0)
Receiving objects: 100% (3/3), done.
